In [7]:
from dotenv import load_dotenv
load_dotenv()
import os 
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec


In [5]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="shaun",dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [6]:
import json 
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Jane Smith',
  'subject': 'Calculus',
  'stars': 4,
  'review': 'Dr. Smith is very knowledgeable and makes complex topics easier to understand.'},
 {'professor': 'Professor John Doe',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Professor Doe has a deep understanding of the subject but sometimes goes too fast.'},
 {'professor': 'Dr. Emily Johnson',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Dr. Johnson is enthusiastic and makes the classes very engaging and informative.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Chemistry',
  'stars': 2,
  'review': "Dr. Brown's lectures are hard to follow, and the exams are very difficult."},
 {'professor': 'Professor Sarah Lee',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Professor Lee is a great instructor with a lot of real-world experience.'},
 {'professor': 'Dr. Robert Wilson',
  'subject': 'History',
  'stars': 3,
  'review': "Dr. Wilson's lectures are informative, but the course materi

In [10]:
processed_data= []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }

    })

In [4]:
processed_data[0]

NameError: name 'processed_data' is not defined

In [12]:
index = pc.Index('shaun')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 21}

In [13]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 21}},
 'total_vector_count': 21}